# Código para controlar una taquilla mediante llaveros RFID


Sistema que gestiona 20 taquillas con acceso mediante llaveros RFID.
Componentes:
- Raspberry pi pico (lo cambiaré por una ESP 32 para que se comunique por ESP-NOW con las centralitas de cada torre de cuatro taquillas)
- Pantalla LCD con placa I2C
- Lector de tarjetas RFID modelo RC522
- Cerrojos accionados eléctricamente. De 12v, normalmente cerrados.
- Relés para accionar los cerrojos

## Pinouts
El proyecto funciona con cualquiera de estos cuatro modelos de placa:
- Raspberry Pi Pico
- RP2040 zero
- ESP32
- ESP32 mini

Raspberry pi pico
![Raspberry pi pico](https://www.geekfactory.mx/wp-content/uploads/raspberry-pi-pico-pinout-1024x660.png.webp)

RP 2040 zero

![RP2040zero](https://www.electronics-lab.com/wp-content/uploads/2025/03/rp2040-pinout.jpg)

ESP32
![ESP32](https://www.upesy.com/cdn/shop/files/doc-esp32-pinout-reference-wroom-devkit.png)




ESP32 mini
![alternative text](https://community.openmqttgateway.com/uploads/default/optimized/2X/4/4000f7bb33b4f3fa619e079742fda1535bab8ba9_2_690x474.png)

ESP32 mini
![alternative text](https://community.openmqttgateway.com/uploads/default/optimized/2X/4/4000f7bb33b4f3fa619e079742fda1535bab8ba9_2_690x474.png)

ESP32 C3 supermini
![text](https://ae-pic-a1.aliexpress-media.com/kf/S61d63cbe55f2482db701eb5027009480Z.jpg_640x640q75.jpg)

## Conexiones

### Pantalla LCD
- Ground a cualquier GND de la placa
- VCC a la salida de 5v de la placa
- SDA al GP14
- SDL al GP15

### Lector RFID RC522
Siguiendo las instrucciones de [Tom's Hardware](https://www.tomshardware.com/raspberry-pi/raspberry-pi-pico/how-to-use-an-rfid-reader-with-a-raspberry-pi-pico):
- SDA - GP1
- SCK - GP2
- MOSI - GP3
- MISO - GP4
- IR - NO SE CONECTA
- GND - CUALQUIER GND
- RST - GP0
- 3.3v - 3V3 OUT

### Cableado de la cerradura de la taquilla
#### Relé
- DC-: al - de la alimentación de 12v (si el relé es es de 12v)
- DC +: al + de la alimentación de 12v (si el relé es de 12v)
- IN: al pin que activa la cerradura (GP16, es la variable "puerta" en el código)
- COM: a DC-
- NO: a la cerradura

### Cerradura
La polaridad **influye**
- Conexión - : a NO del relé
- Conexión + : a + de la fuente de alimentación de 12v

## Módulos necesarios
Hay que copiar estos módulos a la placa Pico/ESP32. Abrir cada enlace, copiar el código y guardarlo en la placa con ese nombre (mfrc522.py, lcd_api.py, pico_i2c_lcd.py)
- mfrc522 [enlace al código](https://raw.githubusercontent.com/kevinmcaleer/pico-rfid/refs/heads/main/mfrc522.py) Permite leer los llaveros RFID
- lcd_api [enlace al código](https://raw.githubusercontent.com/dhylands/python_lcd/master/lcd/lcd_api.py) Pantalla LCD
- pico_i2c_lcd [enlace al código](https://raw.githubusercontent.com/T-622/RPI-PICO-I2C-LCD/refs/heads/main/pico_i2c_lcd.py) Pantalla LCD

## Código

Importamos los módulos

In [ ]:
from mfrc521 import MFRC522
import _thread
import utime
import json
import machine
import math
import machine
from machine import Pin
from machine import I2C
from lcd_api import LcdApi
from pico_i2c_lcd import I2cLcd

Establecemos los parámetros

In [ ]:
#####	PARÁMETROS	#####

reader = MFRC522(spi_id=0,sck=2,miso=4,mosi=3,cs=1,rst=0)
global limite
limite=10
tiempo_puerta=1
tiempo_entre_comprobaciones=1
tarjeta_maestra="[0x72, 0x58, 0x61, 0x51]"

# Parámetros de la puerta
LED = machine.Pin(25,machine.Pin.OUT)
rojo = machine.Pin(18,machine.Pin.OUT)
verde = machine.Pin(17,machine.Pin.OUT)
puerta = machine.Pin(16,machine.Pin.OUT)

# Parámetros de la pantalla
I2C_ADDR     = 0x27    # Decimal 39
I2C_NUM_ROWS = 2
I2C_NUM_COLS = 16
SDA_PIN = 14
SCL_PIN = 15


Definimos las funciones

In [ ]:
#####	FUNCIONES	#####

# Puesta en marcha de la pantalla
i2c = I2C(1, sda=machine.Pin(SDA_PIN),

    scl=machine.Pin(SCL_PIN),

    freq=400000)

lcd = I2cLcd(i2c, I2C_ADDR, I2C_NUM_ROWS, I2C_NUM_COLS)
# lcd.putstr("holaola")
# 
# utime.sleep(200)
# 
# lcd.clear()
# 
# lcd.hide_cursor()
# 
# timer = 0
# 
# running = False

def encender(elemento):
    elemento.value(1)
def apagar(elemento):
    elemento.value(0)
    
def parpadeo(elemento):
    
    for i in range(5):
        elemento.value(1)
        utime.sleep_ms(100) 
        elemento.value(0)
        utime.sleep_ms(100)
        
def liberar_taquilla():
    data={'ocupada': False, 
              'tarjeta' :"",
                   "fecha_inicio":0}
    g=open("archivo", "w")
    g.write(json.dumps(data))
    g.close()
    apagar(rojo)
    encender(verde)
    encender(puerta)
    utime.sleep(tiempo_puerta)
    apagar(puerta)
def ocupar_taquilla(tarjeta_leida):
    data={'ocupada': True, 
              'tarjeta' :tarjeta_leida,
                   "fecha_inicio":utime.time()}
    g=open("archivo", "w")
    g.write(json.dumps(data))
    g.close()
    encender(puerta)
    utime.sleep(tiempo_puerta)
    apagar(puerta)
    apagar(verde)
    encender(rojo)
    
    
def comprobar_tiempo_uso():
    global limite
    while True:
        g=open("archivo", "r")
        data=json.load(g)
        g.close()
        ocupada=data['ocupada']
        tarjeta_guardada=data['tarjeta']
        fecha_inicio=data['fecha_inicio']
        ahora=utime.time()
        if ahora-fecha_inicio>limite and fecha_inicio!=0:
            print("tiempo excedido")
            data={'ocupada': False, 
              'tarjeta' :"",
                   "fecha_inicio":0}
            g=open("archivo", "w")
            g.write(json.dumps(data))
            g.close()
            for i in range(5):
                encender(verde)
                utime.sleep(0.1)
                apagar(verde)
                utime.sleep(0.1)
                encender(puerta)
                utime.sleep(0.1)
                apagar(puerta)
                encender(verde)
        
        utime.sleep(tiempo_entre_comprobaciones)
        

### Instrucciones de arranque previas al inicio del loop de funcionamiento
- Se inicia un proceso en paralelo para que el led que tiene la placa Pico empieze a parpadear, de manera que sepamos que está funcionando.
- Se comprueba que existe el archivo en el que se almacena la información sobre la taquilla (si está ocupada, qué tarjeta tiene asignada, en qué fecha y hora se asignó). Si no existe este archivo, lo crea.
- Abre el archivo y carga la información que contiene. Es un diccionario almacenado en formato json.


In [ ]:
#####	INICIA UN PARPADEO PARA PODER IDENTIFICAR QUE EL PROGRAMA ESTÁ EN MARCHA	#####
#_thread.start_new_thread(parpadeo, ())
        
##### COMPRUEBA SI EXISTE EL ARCHIVO DE PERSISTENCIA	#####
try:
    g=open("archivo", "r")
    data=json.load(g)
    
    g.close()
#####	SI EL ARCHIVO NO EXISTE, LO CREA Y LO PUEBLA	#####
except:
    f=open("archivo", "w")
    diccionario= {'ocupada': False, 
              'tarjeta' :"",
                   "fecha_inicio":0} 
    f.write(json.dumps(diccionario))
    f.close()
#####	ABRE LA PERSISTENCIA Y CARGA LOS VALORES DE LAS VARIABLES	#####
# g=open("archivo", "r")
# data=json.load(g)
# ocupada=data['ocupada']
# tarjeta_guardada=data['tarjeta']
# fecha_inicio=data['fecha_inicio']
# print(ocupada, tarjeta_guardada, fecha_inicio)
# 
# g.close()


## Inicio de los loops de funcionamiento
- Loop que comprueba que no se haya excedido el tiempo de uso de la taquilla. De ser así, la taquilla pasa a estar disponible. Daba problemas. Quizás se deba a que el loop del led ya ocupa uno de los dos núcleos de la placa. Puede que quitando el loop del led se solucione.
- Loop principal. Lee las tarjetas. Asigna la taquilla si está libre, graba los cambios en el archivo de persistencia.